In [1]:
import numpy as np
import pandas as pd
import torch
import torch.utils.data as data_utils
import torch.nn.functional as F
import torch.nn as nn
import os
from sklearn.model_selection import train_test_split

In [2]:
trainFile = "C:/Users/ajit/Desktop/Problem_05.csv"
pwd = os.getcwd()
os.chdir(os.path.dirname(trainFile))
df=pd.read_csv(os.path.basename(trainFile), parse_dates=['Time'])

In [3]:
df_1=df[(df['Turbine']==124) & df['NtfOk']==True]

In [4]:
x=df_1[['RpmB','Energy','WindN']]
y=df_1['WindMfs']

In [5]:
X_train,X_test,y_train,y_test=train_test_split(np.array(x),np.array(y),random_state=5)

In [6]:
X_tr,X_te,y_tr,y_te=torch.from_numpy(X_train),torch.from_numpy(X_test),torch.from_numpy(y_train.reshape(-1,1)),torch.from_numpy(y_test.reshape(-1,1))

In [7]:
trainset=data_utils.TensorDataset(X_tr.float(),y_tr.float())
trainloader=data_utils.DataLoader(trainset,batch_size=512,shuffle=True)
testset=data_utils.TensorDataset(X_te.float(),y_te.float())
testloader=data_utils.DataLoader(testset,batch_size=128)

In [32]:
class pytorch_model(nn.Module):
    
    def __init__(self,input_size,first_size,second_size,third_size,output_size):
        super().__init__()
        self.fc1=nn.Linear(input_size,first_size)
        self.fc2=nn.Linear(first_size,second_size)
        self.fc3=nn.Linear(second_size,third_size)
        self.fc4=nn.Linear(third_size,output_size)
    def forward(self,x):
        l1=F.relu(self.fc1(x))
        l2=F.relu(self.fc2(l1))
        l3=F.relu(self.fc3(l2))
        out=self.fc4(l3)
        return out

In [33]:
model=pytorch_model(3,8,5,5,1)

In [34]:
criteria=nn.MSELoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)

In [35]:
for epoch in range(10):
    running_loss=0.0
    for i,data in enumerate(trainloader):
        inputs,labels=data
        outputs=model(inputs)
        loss=criteria(outputs,labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss+=loss
        if i %8==7:
            print('[%d, %5d] loss: %.3f' % (epoch+1,i+1,running_loss/8))
            running_loss=0.0
print('Finished training')

[1,     8] loss: 45.269
[1,    16] loss: 33.058
[1,    24] loss: 23.900
[1,    32] loss: 14.650
[1,    40] loss: 9.941
[2,     8] loss: 7.357
[2,    16] loss: 6.315
[2,    24] loss: 6.519
[2,    32] loss: 6.698
[2,    40] loss: 6.058
[3,     8] loss: 5.861
[3,    16] loss: 6.319
[3,    24] loss: 5.926
[3,    32] loss: 5.910
[3,    40] loss: 5.628
[4,     8] loss: 5.331
[4,    16] loss: 5.461
[4,    24] loss: 5.686
[4,    32] loss: 5.343
[4,    40] loss: 4.997
[5,     8] loss: 4.937
[5,    16] loss: 4.997
[5,    24] loss: 4.656
[5,    32] loss: 4.701
[5,    40] loss: 3.977
[6,     8] loss: 4.268
[6,    16] loss: 3.985
[6,    24] loss: 3.728
[6,    32] loss: 3.750
[6,    40] loss: 3.179
[7,     8] loss: 3.241
[7,    16] loss: 3.044
[7,    24] loss: 2.762
[7,    32] loss: 2.401
[7,    40] loss: 2.773
[8,     8] loss: 2.420
[8,    16] loss: 2.572
[8,    24] loss: 2.817
[8,    32] loss: 2.290
[8,    40] loss: 2.376
[9,     8] loss: 2.459
[9,    16] loss: 2.465
[9,    24] loss: 2.194
[9,    

In [37]:
test_output=model(X_te.float())

In [41]:
(torch.sum((test_output-y_te.float())**2))/len(y_te)

tensor(2.2822)